# SI 650 / EECS 549: Homework 3 Part 2

Homework 3 Part 2 will have you working with Learning to Rank approaches, which as we've seen are highly competitive when enough data is available. Part 2 should be completed after Part 1 and will use some small parts of that code to construct an index and load data.

If you haven't worked with them before, Part 2 will expose you to [overloaded python operators](https://www.geeksforgeeks.org/operator-overloading-in-python/), which will let you call functions using alternative python syntax. For example, if you're using two Python `list` objects, you can interact with them using `+` to call the equivalent method. 

In Part 2, you'll work on the following tasks:
 - Construct and learn learning-to-rank pipelines
 - Add new features for learning-to-rank
 - Evaluate learning-to-rank models
 
Part 2 can be run on most laptops as well. Some of the machine learning models may take a few minutes to train but you are also welcome to run these on Great Lakes. None of the code in this part involves deep learning or requires a GPU.

### Launch PyTerrier
Import the packages you need and launch PyTerrier like we did in Part 1. You will still need to make sure `JAVA_HOME` is set for this part.

In [ ]:
import pyterrier as pt
import pandas as pd
import os

In [ ]:
if not pt.started():
    pt.init()

# Indexing CORD19

Like in Part 1, we'll again use the CORD19 data. However, **in Part 2, we will add in positional indexing.** This code will still look very similar to your code from Part 1. When creating the index, add the `blocks=True` argument, which will include word order information in the index. On most laptops, this process will take ~1 minute.

You may see an error `java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755` during indexing but you can safely ignore the error for the purposes of this notebook.

In [ ]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_trec_covid_positional_indices'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 

    # TODO


    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    
    # TODO
    pass

else:
    # if you already have the index, use it.
    
    # TODO
    pass

index = pt.IndexFactory.of(index_ref)

## Transformers and Operators

PyTerrier works extensively with objects/functions that _transform_ one input to another. We saw this behavior with the `BatchRetrieve` object that we used earlier to get search results, which had a `transform()` method that takes as input a dataframe, and returns another dataframe. We can think of this function as a *transformation* of the earlier dataframe (e.g., transforming the queries into results). PyTerrier has many such functions that act as  [transformers](https://pyterrier.readthedocs.io/en/latest/transformer.html). 

Let's use the transformer, `BatchRetrieve`, and this time specify that we'll use the TF-IDF word model.

In [ ]:
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")

In PyTerrier, all transformers have been coded so that they be combined using Python operators, which is an example  of operator overloading. If we want to have the output of one transformer used as the input of another transformer, we can use the `>>` operator. This operator lets us compose a series of transformations to output our document rankings.

In [ ]:
# This is our first retrieval transformer, which transforms a queries dataframe to a results dataframe
tf = pt.BatchRetrieve(index, wmodel="Tf")

tf( cord19.get_topics(variant='title').head(1) )

Let's define our first pipeline using the `>>` operator. If we have two transformers, `a` and `b`, the result of `a >> b` is itself a transformer!

In [ ]:
pipeline = tf >> tfidf

We can use this pipeline like any other transformer. Here, we'll  pass in the very first query using `head(1)` and get the results

In [ ]:
pipeline(cord19.get_topics(variant='title').head(1))

There many other PyTerrier operators and please see examples in the [PyTerrier documentation on operators](https://pyterrier.readthedocs.io/en/latest/operators.html)

## Task 1: Pipeline Construction (15 ponts)

Create a ranker that performs the following:
 - obtains the top 10 highest scoring documents by term frequency (`wmodel="Tf"`)
 - obtains the top 10 highest scoring documents by TF.IDF (`wmodel="TF_IDF"`)
 - reranks only those documents found in BOTH of the previous retrieval settings using BM25.

How many documents are retrieved by this full pipeline for the query `"chemical"`.
> If you obtain the correct solution, the document with docno `"37771"` should have a score of $12.426309	$ for query `"chemical"`.

Hints:
 - choose carefully your [PyTerrier operators](https://pyterrier.readthedocs.io/en/latest/operators.html)
 - you should not need to perform any Pandas dataframe operations

In [ ]:
# TODO

## Developing more complex transformers pipelines

PyTerrier has a number of useful transformers that we can use to create complex retrieval pipelines. For composing pipelines, let's first define a bit of notation:
 - $Q$: a set of queries
 - $D$: a set of documents
 - $R$: a set of retrieved documents for a set of queries

In our setting, we'll use three transformers:
 - `pt.BatchRetrieve(index, wmodel="BM25")` - input $Q$ or $R$ (retrieval or reranking), output $R$
 - `pt.rewrite.SDM()` (sequential dependence proximity model) - input $Q$, output $Q$. 
 - `pt.rewrite.Bo1QueryExpansion(index)` - input $R$, output $Q$.

Note that now, we're using BM25 instead of TF-IDF to retrieve documents.

Transformers like `SDM` are performing query augmentation. Here, `SDM` is reweighting terms using a Dirichlet language model like we talked about in class. However, many query rewrites a possible! For example, we could use WordNet to expand the query with synonyms or even define our own. 

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
sdm = pt.rewrite.SDM()
qe = pt.rewrite.Bo1QueryExpansion(index)

Let's see how `sdm` applies to a given query. This generates a query in an [Indri-like query language](https://www.lemurproject.org/lemur/IndriQueryLanguage.php) that Terrier (cf. `pt.BatchRetrieve()`) can understand.
 - `#combine()` - is used for weighting sub-expressions
 - `#1() - matches as a phrase, i.e. how many times do the constituent words exactly match as a phrase
 - `#uw8()` and `#uw12()` look for how many times the constituent words appear in unordered windows of 8 or 12 tokens.
 - finally, the weighting model is overridden for these query terms.
  

In [ ]:
sdm.search("chemical reactions").iloc[0]["query"]

## Task 2.1: Creating Experiments to test Pipelines (3 points)

Which kinds of rankers will perform better? We can answer this by creating an [Experiment](https://pyterrier.readthedocs.io/en/latest/experiments.html) that will compare sequential dependence model and Bo1 query expansion on TREC CORD19 with the BM25 baseline. We used Experiments in Part 1, so we'll expand this idea again here to test out different pipelines.

**Your Task:** You will need to construct appropriate pipelines to compare each approach by considering the input and output datatypes of the `bm25`, `sdm` and `qe`. 

In [ ]:
topics = cord19.get_topics(variant='title')
qrels = cord19.get_qrels()

In [ ]:
# TODO

### Task 2.2:  Reflection (2 points)

Which approaches result in significant increases in NDCG and MAP? Is NDCG@10 also improved? Write 2-3 sentences why you think this is the case.

# Learning to Rank

Now that we have some idea how to build pipelines in PyTerrier, we'll continue our exploration by constructing and evaluating Learning to Rank pipelines.

Learning to rank is fundamentally a machine learning approach to IR. To work, we'll need to create _training data_ to learn our ranking model. We'll also want development ("dev") data to evaluate hyperparameters and test data to see how well it works. Our dataset doesn't by itself already have the data split into train, dev, and test, so we'll first need to do that.

TREC Covid only has 50 topics, which isn't a lot for training in general. However, for a homework, 50 is sufficient to show you how it works. In this case, we'll split this 30 for training (60%), 5 for validation (10%), and 15 for evaluation (30%). In your projects, you'll want to test for statistical significance (i.e., is the improvement better than what could be expected by chance?) so we will also examine statistical significance, though we have limited data (15 topics) to do this with.

When training our model, we will only re-rank the top 500 documents for each query, which is hopefully sufficient data for learning to rank. We don't set the model to rerank _all_ documents (though you can try this!) since it causes us to have to train the Learning to Rank model on all data and is often very slow in practice.

In [ ]:
RANK_CUTOFF = 10
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

## Defining a Feature Set for Learning to Rank

Learning to Rank requires that we define which features to use in determining a ranking. We can use scores like BM25 but it's common to try adding more complex features that take advantage of what you, the IR practitioner, knows about the data or features that would be useful.

For this part, we'll start with 7 features. PyTerrier provides [extensive support](https://pyterrier.readthedocs.io/en/latest/ltr.html#) for how to define features and calculate them for Learning to Rank. We recommend reading that documentation as you start to go over the code below. Note that several of these features will use more than just the text to compute relevance!

1.   the BM25 abstract score;
2.   sequential dependence model, scored by BM25;
3.   does the abstract contain 'coronavirus covid', scored by BM25;
4.   the BM25 score on the title (even though we didn't index it earlier!);
5.   was the paper released/published in 2020? Recent papers were more useful for this task;
6.   does the paper have a DOI, i.e. is it a formal publication?
7.   the coordinate match score for the query--i.e. how many query terms appear in the abstract.

Several of these feature require additional metadata `["title", "date", "doi"]`, which is present in the TREC covid dataset. Fortunately, we can obtain this metadata after indexing using `pt.text.get_text(cord19, ["title", "date", "doi"])` to retrieve these extra metadata columns from the original data.

There is a lot going on in this code block, but it's useful to think of this as _another_ example of operator overloading in PyTerrier. Here, we're combining features using the `**` operator, which is the [feature union](https://pyterrier.readthedocs.io/en/latest/ltr.html#calculating-features) operator in PyTerrier. 

The output of our feature union is the `ltr_feats1` object which is itself a transformer we can use to start ranking.

In [ ]:
ltr_feats1 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(cord19, ["title", "date", "doi"]) >> (
    pt.transformer.IdentityTransformer()
    ** # sequential dependence
    (sdm >> bm25)
    ** # score of text for query 'coronavirus covid'
    (pt.apply.query(lambda row: 'coronavirus covid') >> bm25)
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="title", takes='docs', wmodel='BM25') ) 
    ** # date 2020
    (pt.apply.doc_score(lambda row: int("2020" in row["date"])))
    ** # has doi
    (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    ** # abstract coordinate match
    pt.BatchRetrieve(index, wmodel="CoordinateMatch")
)

# for reference, lets record the feature names here too
fnames=["BM25", "SDM", 'coronavirus covid', 'title', "2020", "hasDoi", "CoordinateMatch"]

To get a sense of what this process is doing, let's look at the output for the query "coronavirus origin". We can see that we now have extra document metadata columns `["title", "date", "doi"]`, as well as the `"features"` column, which is what we'll use for learning. 

In [ ]:
ltr_feats1.search("coronovirus origin")

We can also look at the raw feature values themselves too. Here, we'll look at the features for the first document. Note that the BM25 in the "score" column above is also the first value in the feature array (20.54), because we used an identity transformer, which returns the same value as its input.

In [ ]:
ltr_feats1.search("coronovirus origin").iloc[0]["features"]

## Actually Doing the Learning for Learning to Rank

In class, we talked about three types of Learning to rank: pointwise, pairwise, and listwise. Here, we'll train a few models for two of these types, listwise and pointwise. 

 - coordinate ascent from FastRank, a listwise linear technique
 - random forests from `scikit-learn`, a pointwise regression tree technique
 - LambdaMART from LightGBM, a listwise regression tree technique

We can use the same feature pipeline, `ltr_feats1`, with all three.  To train the ranker, we compose it (`>>`) with the learned model. We use `pt.ltr.apply_learned_model()` which knows how to deal with different learners.

The full pipeline is then fitted (learned) using `.fit()`, specifying the training topics and qrels, much like how we train models with Scikit-Learn. Importantly, the preceding stages of the pipeline (retrieval and feature calculation) are applied to the training topics in order to obtain the results, which are then passed to the learning to rank technique. LightGBM has early stopping enabled, which uses a validation topics set--similarly the validation topics are transformed into validation results.

Finally, `%time` is notebook "magic command" specific to Juypyter which displays how long learning takes for each technique. Learning for each technique takes under 30 seconds.

In [ ]:
import fastrank

train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(train_request, form='fastrank')

%time ca_pipe.fit(train_topics, cord19.get_qrels())

Now, let's try learning a different pointwise model to estimate relevance scores using a Random Forest model from Scikit-Learn. This is a good example of how we can use off-the-shelf regression models in PyTerrier to perform learning to rank.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=2)

rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(rf)

%time rf_pipe.fit(train_topics, cord19.get_qrels())

Finally, let's use a list-wise approach, which typically performs best among the three types of Learning to Rank

In [ ]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, cord19.get_qrels(), valid_topics, cord19.get_qrels())

## Evaluating all the solutions 

We've fit the three Learning to Rank models so we can now use them to predict relevance scores for the held-out 15 topics (queries) in our test set. Note that if we wanted to do any tuning (e.g., pick the number of leaves in our `RandomForestRegressor`, we could run this evaluation on our held-out _dev_ set and pick the hyperparameters that produced best performance on it).

### Task 3: Create an experiment to compare (5 points)

Create a new `Experiment` to compare our three solutions and BM25 at the `RANK_CUTOFF` (four models in total). Report MAP, NDCG, and NDCG@10 measures and one new measure: mean response time (`"mrt"`), which will tell us how fast the models are able to score documents--something important if we're going to use them in the real world!

In [ ]:
# TODO

Thats really interesting – all three learned models could improve NDCG@10 over BM25, but the coordinate ascent model improved the most (significantly so on all three metrics, but again on only 15 queries). Coordinate Ascent improved upto 10 queries.

## Analysis: What Features Matter?

Since we're _learning_ a ranking model, we can inspect that model to see what information it's using. One option for inspecting the model is to evaluate the performance of each feature independently to see how much it contributes to the eventual relevance. To examine each feature, we will create separate a separate model from each feature in our feature pipeline (`ltr_feats1`) by using `pt.ltr.feature_to_score(i)` with a particular feature number $i$. In essence, this will only score the relevance using a single feature's values, even though we've already calculated all of them.

In [ ]:
pt.Experiment(
    [ltr_feats1 >> pt.ltr.feature_to_score(i) for i in range(len(fnames))],
    test_topics,
    qrels, 
    names=fnames,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "num_rel_ret"])

### Inspecting the models

To get a sense of feature importance, we can also analyze the learned models themselves using their internal feature weights in the Scikit-Learn models. Here, we'll plot the feature importances in a few ways. For the coordinate ascent model, we plot the feature weights (note the log-scale y-axis); while for the regression-tree based techniques, we report the feature importance.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt, numpy as np

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(10, 6))

ax0.bar(np.arange(len(fnames)), ca_pipe[1].model.to_dict()['Linear']['weights'])
ax0.set_xticks(np.arange(len(fnames)))
ax0.set_xticklabels(fnames, rotation=45, ha='right')
ax0.set_title("Coordinate Ascent\n Feature Weights")
ax0.set_yscale('log')

ax1.bar(np.arange(len(fnames)), rf.feature_importances_)
ax1.set_xticks(np.arange(len(fnames)))
ax1.set_xticklabels(fnames, rotation=45, ha='right')
ax1.set_title("Random Forests\n Feature Importance")

ax2.bar(np.arange(len(fnames)), lmart_l.feature_importances_)
ax2.set_xticks(np.arange(len(fnames)))
ax2.set_xticklabels(fnames, rotation=45, ha='right')
ax2.set_title("$\lambda$MART\n Feature Importance")

fig.show()

## Task 4: Adding new features (15 points)

We've seen how to develop Learning to Rank models and analyze features. Your task is the following:

  - Define _at least_ two additional features to use in learning to rank models. These can be functions that you define or additional options from PyTerrier
  - Include your new features in a new learning to rank pipeline using the three approaches we have above to create three new models
  - Evaluate your new models against the old models in an Experiment
 
**Full credit will depend on one of your new models outperforming the best of the initial three models.**

You should aim to evaluate your features initially on the `dev` set, rather than the `test` set in order to avoid overfitting your features (i.e., never look at the test set until the end). You can use the feature and model inspection code to give you some idea of what the model is looking at. We also recommend looking at the data directly to think about what kinds of features might be useful

In [ ]:
# TODO